<a href="https://colab.research.google.com/github/AfterMaturi/KensakuGijutsu/blob/main/Kensaku.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1.データ**

  青空文庫の文章データをURLからファイルを開きダウンロードして使用する。

**2.分析手法、**

ward2vecを用いて夏目漱石、太宰治、芥川龍之介、森鷗外の小説用いてを類似単語を調べる。
ward2vecとは自然言語処理の手法で、文章中の単語を意味を示す数値ベクトルに変換し、その意味を把握するものである。
最も小説を処理することに適しているので、この手法を選択する。

In [63]:
#1:インポート
import re
import zipfile
import urllib.request
import os.path
import glob

!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.7

import MeCab

from sklearn.feature_extraction.text import TfidfVectorizer

import pandas as pd

Reading package lists... Done
Building dependency tree       
Reading state information... Done
aptitude is already the newest version (0.8.10-6ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 48 not upgraded.
mecab is already installed at the requested version (0.996-5)
libmecab-dev is already installed at the requested version (0.996-5)
mecab-ipadic-utf8 is already installed at the requested version (2.7.0-20070801+main-1)
git is already installed at the requested version (1:2.17.1-1ubuntu0.12)
make is already installed at the requested version (4.1-9.1ubuntu1)
curl is already installed at the requested version (7.58.0-2ubuntu3.19)
xz-utils is already installed at the requested version (5.2.2-1.3ubuntu0.1)
file is already installed at the requested version (1:5.32-2ubuntu0.4)
mecab is already installed at the requested version (0.996-5)
libmecab-dev is already installed at the requested version (0.996-5)
mecab-ipadic-utf8 is already installed at the requested version (2.7.0-2

In [64]:
!pip install janome
from janome.tokenizer import Tokenizer
from gensim.models import word2vec

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [65]:
#2:テキストファイルの読み込み

#夏目漱石・『吾輩は猫である』『三四郎』『こころ』
NS_URL1 = 'https://www.aozora.gr.jp/cards/000148/files/789_ruby_5639.zip'
NS_URL2 =  'https://www.aozora.gr.jp/cards/000148/files/794_ruby_4237.zip'
NS_URL3 ='https://www.aozora.gr.jp/cards/000148/files/773_ruby_5968.zip' 

In [66]:
#太宰治・『人間失格』『津軽』『女生徒』
DO_URL1 = 'https://www.aozora.gr.jp/cards/000035/files/301_ruby_5915.zip'
DO_URL2 = 'https://www.aozora.gr.jp/cards/000035/files/2282_ruby_1996.zip'
DO_URL3 = 'https://www.aozora.gr.jp/cards/000035/files/275_ruby_1532.zip' 

In [67]:
#芥川龍之介・『羅生門』『歯車』『鼻』
AR_URL1 = 'https://www.aozora.gr.jp/cards/000879/files/127_ruby_150.zip'
AR_URL2 = 'https://www.aozora.gr.jp/cards/000879/files/42377_ruby_34744.zip'
AR_URL3 = 'https://www.aozora.gr.jp/cards/000879/files/42_ruby_154.zip'

In [181]:
#森鷗外・『舞姫』『高瀬舟』『ヰタ・セクスアリス』
MO_URL1='https://www.aozora.gr.jp/cards/000129/files/58126_ruby_73643.zip'
MO_URL2='https://www.aozora.gr.jp/cards/000129/files/45245_ruby_21882.zip'
MO_URL3='https://www.aozora.gr.jp/cards/000129/files/695_ruby_22805.zip'

In [68]:
#3ドキュメント取得
def download(URL):
    # zipファイルのダウンロード
    zip_file = re.split(r'/', URL)[-1]
    urllib.request.urlretrieve(URL, zip_file)
    dir = os.path.splitext(zip_file)[0]

    # zipファイルの解凍と保存
    with zipfile.ZipFile(zip_file) as zip_object:
        zip_object.extractall(dir)

    os.remove(zip_file)

    # 保存したファイルのパスを取得
    path = os.path.join(dir,'*.txt')
    list = glob.glob(path)
    return list[0]

In [69]:
#4文の抽出・整形
def convert(download_text):
    # ファイル読み込み
    data = open(download_text, 'rb').read()
    text = data.decode('shift_jis')

    # 本文の抽出
    text = re.split(r'\-{5,}', text)[2]  
    text = re.split(r'底本：', text)[0]
    text = re.split(r'［＃改ページ］', text)[0]

    # ノイズ削除
    text = re.sub(r'《.+?》', '', text)
    text = re.sub(r'［＃.+?］', '', text)
    text = re.sub(r'｜', '', text)
    text = re.sub(r'\r\n', '', text)
    text = re.sub(r'\u3000', '', text)  
    text = re.sub(r'「', '', text) 
    text = re.sub(r'」', '', text)
    text = re.sub(r'、', '', text)
    text = re.sub(r'。', '', text)

    return text

In [70]:
file = download(NS_URL1)
NS_text1 = convert(file)

file = download(NS_URL2)
NS_text2 = convert(file)

file = download(NS_URL3)
NS_text3 = convert(file)

NS_text = NS_text1 + NS_text2 + NS_text3

In [71]:
file = download(DO_URL1)
DO_text1 = convert(file)

file = download(DO_URL2)
DO_text2 = convert(file)

file = download(DO_URL3)
DO_text3 = convert(file)

DO_text = DO_text1 + DO_text2 + DO_text3

In [72]:
file = download(AR_URL1)
AR_text1 = convert(file)

file = download(AR_URL2)
AR_text2 = convert(file)

file = download(AR_URL3)
AR_text3 = convert(file)

AR_text = AR_text1 + AR_text2 + AR_text3

In [182]:
file = download(MO_URL1)
MO_text1 = convert(file)

file = download(MO_URL2)
MO_text2 = convert(file)

file = download(MO_URL3)
MO_text3 = convert(file)

MO_text = MO_text1 + MO_text2 + MO_text3

In [195]:
#5 形態素解析 --- (※3)
t = Tokenizer()
results = []
# テキストを一行ずつ処理する
lines = NS_text.split("\r\n")
for line in lines:
    s = line
    s = s.replace('｜', '')
    s = re.sub(r'《.+?》', '', s) # ルビを削除
    s = re.sub(r'［＃.+?］', '', s) # 入力注を削除
    tokens = t.tokenize(s) # 形態素解析
    # 必要な語句だけを対象とする --- (※4)
    r = []
    for tok in tokens:
        if tok.base_form == "*": # 単語の基本系を採用
            w = tok.surface
        else:
            w = tok.base_form
        ps = tok.part_of_speech # 品詞情報
        hinsi = ps.split(',')[0]
        if hinsi in ['名詞', '形容詞', '動詞', '記号']:
            r.append(w)
    rl = (" ".join(r)).strip()
    results.append(rl)
    #print(rl) # --- 画面に分かち書きした行を表示

    # 書き込み先テキストを開く --- (※5)
wakati_file = '夏目漱石b.wakati'
with open(wakati_file, 'w', encoding='utf-8') as fp:
    fp.write("\n".join(results))

# Word2Vecでモデルを作成 --- (※6)
data = word2vec.LineSentence(wakati_file)
model = word2vec.Word2Vec(data, 
    size=200, window=10, hs=1, min_count=2, sg=1)
model.save('夏目漱石b.model')

model = word2vec.Word2Vec.load('夏目漱石b.model')

for word in ['結婚', '幸','人生','友']:
    words = model.most_similar(positive=[word])
    n = [w[0] for w in words]
    print(word,"=", ",".join(n))

結婚 = 生まれつく,弁論,諾否,聞き及ぶ,差異,祝,穏当,不可能,予審,集散
幸 = 訳合,にがい,夜毎,こだわり,むると,取りとめる,機能,瀾怒濤,意思,此種
人生 = 進捗,口舌,争論,願望,嗜欲,ムード,閑生涯,有意義,厳粛,教訓
友 = おくる,ヴァレリアス,江湖,揶揄,面前,せり,一眼,処士,もとづく,陋策


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).


In [196]:
t = Tokenizer()
results = []
# テキストを一行ずつ処理する
lines = DO_text.split("\r\n")
for line in lines:
    s = line
    s = s.replace('｜', '')
    s = re.sub(r'《.+?》', '', s) # ルビを削除
    s = re.sub(r'［＃.+?］', '', s) # 入力注を削除
    tokens = t.tokenize(s) # 形態素解析
    # 必要な語句だけを対象とする --- (※4)
    r = []
    for tok in tokens:
        if tok.base_form == "*": # 単語の基本系を採用
            w = tok.surface
        else:
            w = tok.base_form
        ps = tok.part_of_speech # 品詞情報
        hinsi = ps.split(',')[0]
        if hinsi in ['名詞', '形容詞', '動詞', '記号']:
            r.append(w)
    rl = (" ".join(r)).strip()
    results.append(rl)
    #print(rl) # --- 画面に分かち書きした行を表示

    # 書き込み先テキストを開く --- (※5)
wakati_file = '太宰治b.wakati'
with open(wakati_file, 'w', encoding='utf-8') as fp:
    fp.write("\n".join(results))

# Word2Vecでモデルを作成 --- (※6)
data = word2vec.LineSentence(wakati_file)
model = word2vec.Word2Vec(data, 
    size=200, window=10, hs=1, min_count=2, sg=1)
model.save('太宰治b.model')

model = word2vec.Word2Vec.load('太宰治b.model')

for word in ['結婚', '成長', '幸','人生','友']:
    words = model.most_similar(positive=[word])
    n = [w[0] for w in words]
    print(word,"=", ",".join(n))

結婚 = 不承知,ときめかす,営む,生き抜く,内福,誤解,著述,鍛,世継,うしなう
成長 = 困惑,ゆき,体,いたたまらない,よこしま,時時,理性,くるしむ,掴める,与太者
幸 = 鼓腹,山の幸,撃壌,宿賃,別天地,山麓,ワラビゼンマイウドタケノコフキアザミキノコ,めぐまれる,新浜,出港
人生 = 切実,受難,かがやかしい,指針,既婚,名案,懐疑,のる,片すみ,二目
友 = セリヌンティウス,面前,竹馬,石工,きかせる,ディオニス,佳い,齧る,奸佞,宿す


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:40: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).


In [197]:
t = Tokenizer()
results = []
# テキストを一行ずつ処理する
lines = AR_text.split("\r\n")
for line in lines:
    s = line
    s = s.replace('｜', '')
    s = re.sub(r'《.+?》', '', s) # ルビを削除
    s = re.sub(r'［＃.+?］', '', s) # 入力注を削除
    tokens = t.tokenize(s) # 形態素解析
    # 必要な語句だけを対象とする --- (※4)
    r = []
    for tok in tokens:
        if tok.base_form == "*": # 単語の基本系を採用
            w = tok.surface
        else:
            w = tok.base_form
        ps = tok.part_of_speech # 品詞情報
        hinsi = ps.split(',')[0]
        if hinsi in ['名詞', '形容詞', '動詞', '記号']:
            r.append(w)
    rl = (" ".join(r)).strip()
    results.append(rl)
    #print(rl) # --- 画面に分かち書きした行を表示

    # 書き込み先テキストを開く --- (※5)
wakati_file = '芥川龍之介b.wakati'
with open(wakati_file, 'w', encoding='utf-8') as fp:
    fp.write("\n".join(results))

# Word2Vecでモデルを作成 --- (※6)
data = word2vec.LineSentence(wakati_file)
model = word2vec.Word2Vec(data, 
    size=200, window=10, hs=1, min_count=2, sg=1)
model.save('芥川龍之介b.model')

model = word2vec.Word2Vec.load('芥川龍之介b.model')

for word in ['結婚','人生']:
    words = model.most_similar(positive=[word])
    n = [w[0] for w in words]
    print(word,"=", ",".join(n))

結婚 = 調節,性慾,有効,披露,相談,つらなる,晩餐,しかる,社,慰安
人生 = オリムピツク,主催,大会,踏み入る,胎内,踏み止まる,創痍,泳げる,競技,闘


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:40: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).


In [184]:
t = Tokenizer()
results = []
# テキストを一行ずつ処理する
lines = MO_text.split("\r\n")
for line in lines:
    s = line
    s = s.replace('｜', '')
    s = re.sub(r'《.+?》', '', s) # ルビを削除
    s = re.sub(r'［＃.+?］', '', s) # 入力注を削除
    tokens = t.tokenize(s) # 形態素解析
    # 必要な語句だけを対象とする --- (※4)
    r = []
    for tok in tokens:
        if tok.base_form == "*": # 単語の基本系を採用
            w = tok.surface
        else:
            w = tok.base_form
        ps = tok.part_of_speech # 品詞情報
        hinsi = ps.split(',')[0]
        if hinsi in ['名詞', '形容詞', '動詞', '記号']:
            r.append(w)
    rl = (" ".join(r)).strip()
    results.append(rl)
    #print(rl) # --- 画面に分かち書きした行を表示

    # 書き込み先テキストを開く --- (※5)
wakati_file = '森鷗外b.wakati'
with open(wakati_file, 'w', encoding='utf-8') as fp:
    fp.write("\n".join(results))

# Word2Vecでモデルを作成 --- (※6)
data = word2vec.LineSentence(wakati_file)
model = word2vec.Word2Vec(data, 
    size=200, window=10, hs=1, min_count=2, sg=1)
model.save('森鷗外b.model')

model = word2vec.Word2Vec.load('森鷗外b.model')

for word in ['結婚', '幸','人生','友']:
    words = model.most_similar(positive=[word])
    n = [w[0] for w in words]
    print(word,"=", ",".join(n))

結婚 = dub,余計,秘密,霊,むつかしい,男女,穢い,兼,仲間はずれ,方角
幸 = 年長,理想,鋭敏,疑惑,庄,二つ,ことわる,助,くい,退学
人生 = 心理,有す,研究,情熱,発揮,萌芽,異常,象,状態,芸術
友 = 報じる,嫉む,嘲る,出入り,のち,宣う,途,望み,彼ら,鏡


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:40: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).


各文豪に対して、代表3作を用いて、類似単語を求めたが、それぞれ、類似と言うにはかけ離れていたり、作品の雰囲気を反映しすぎている。
選択した語が登場しないので、分析することができないばあいもあった。

精度を上げるために、それぞれ15作ほどを入力し、類似単語を調べる。

In [78]:
#夏目漱石
NS_URL = ['https://www.aozora.gr.jp/cards/000148/files/789_ruby_5639.zip',
          'https://www.aozora.gr.jp/cards/000148/files/794_ruby_4237.zip',
          'https://www.aozora.gr.jp/cards/000148/files/773_ruby_5968.zip',
          'https://www.aozora.gr.jp/cards/000148/files/752_ruby_2438.zip',
          'https://www.aozora.gr.jp/cards/000148/files/777_ruby_1717.zip',
          'https://www.aozora.gr.jp/cards/000148/files/776_ruby_6020.zip',
          'https://www.aozora.gr.jp/cards/000148/files/775_ruby_2064.zip',
          'https://www.aozora.gr.jp/cards/000148/files/767_ruby_3256.zip',
          'https://www.aozora.gr.jp/cards/000148/files/56143_ruby_50824.zip',
          'https://www.aozora.gr.jp/cards/000148/files/798_ruby_2413.zip',
          'https://www.aozora.gr.jp/cards/000148/files/56923_ruby_66668.zip',
          'https://www.aozora.gr.jp/cards/000148/files/783_ruby_1311.zip',
          'https://www.aozora.gr.jp/cards/000148/files/799_ruby_6024.zip',
          'https://www.aozora.gr.jp/cards/000148/files/1076_ruby_4527.zip',
          'https://www.aozora.gr.jp/cards/000148/files/780_ruby_569.zip']
          
for i in NS_URL:
    download_file = download(i)
    text = convert(download_file)
    NS_text += text

In [114]:
t = Tokenizer()
results = []
# テキストを一行ずつ処理する
lines = NS_text.split("\r\n")
for line in lines:
    s = line
    s = s.replace('｜', '')
    s = re.sub(r'《.+?》', '', s) # ルビを削除
    s = re.sub(r'［＃.+?］', '', s) # 入力注を削除
    tokens = t.tokenize(s) # 形態素解析
    # 必要な語句だけを対象とする --- (※4)
    r = []
    for tok in tokens:
        if tok.base_form == "*": # 単語の基本系を採用
            w = tok.surface
        else:
            w = tok.base_form
        ps = tok.part_of_speech # 品詞情報
        hinsi = ps.split(',')[0]
        if hinsi in ['名詞', '形容詞', '動詞', '記号']:
            r.append(w)
    rl = (" ".join(r)).strip()
    results.append(rl)
    #print(rl) # --- 画面に分かち書きした行を表示

    # 書き込み先テキストを開く --- (※5)
wakati_file = '夏目漱石.wakati'
with open(wakati_file, 'w', encoding='utf-8') as fp:
    fp.write("\n".join(results))

# Word2Vecでモデルを作成 --- (※6)
data = word2vec.LineSentence(wakati_file)
model = word2vec.Word2Vec(data, 
    size=200, window=10, hs=1, min_count=2, sg=1)
model.save('夏目漱石.model')

model = word2vec.Word2Vec.load('夏目漱石.model')


In [142]:
#太宰治
DO_URL = ['https://www.aozora.gr.jp/cards/000035/files/301_ruby_5915.zip',
          'https://www.aozora.gr.jp/cards/000035/files/2282_ruby_1996.zip',
          'https://www.aozora.gr.jp/cards/000035/files/275_ruby_1532.zip',
          'https://www.aozora.gr.jp/cards/000035/files/1574_ruby_15426.zip',
          'https://www.aozora.gr.jp/cards/000035/files/260_ruby_34636.zip',
          'https://www.aozora.gr.jp/cards/000035/files/258_ruby_20178.zip',
          'https://www.aozora.gr.jp/cards/000035/files/1567_ruby_4948.zip',
          'https://www.aozora.gr.jp/cards/000035/files/312_ruby_2924.zip', 
          'https://www.aozora.gr.jp/cards/000035/files/263_ruby_20160.zip',
          'https://www.aozora.gr.jp/cards/000035/files/307_ruby_3042.zip',
          'https://www.aozora.gr.jp/cards/000035/files/46597_ruby_24666.zip',
          'https://www.aozora.gr.jp/cards/000035/files/45675_ruby_20802.zip',
          'https://www.aozora.gr.jp/cards/000035/files/1565_ruby_8220.zip',
          'https://www.aozora.gr.jp/cards/000035/files/46599_ruby_24668.zip',
          'https://www.aozora.gr.jp/cards/000035/files/45676_ruby_20801.zip']

for i in DO_URL:
    download_file = download(i)
    text = convert(download_file)
    DO_text += text

In [143]:
t = Tokenizer()
results = []
# テキストを一行ずつ処理する
lines = DO_text.split("\r\n")
for line in lines:
    s = line
    s = s.replace('｜', '')
    s = re.sub(r'《.+?》', '', s) # ルビを削除
    s = re.sub(r'［＃.+?］', '', s) # 入力注を削除
    tokens = t.tokenize(s) # 形態素解析
    # 必要な語句だけを対象とする --- (※4)
    r = []
    for tok in tokens:
        if tok.base_form == "*": # 単語の基本系を採用
            w = tok.surface
        else:
            w = tok.base_form
        ps = tok.part_of_speech # 品詞情報
        hinsi = ps.split(',')[0]
        if hinsi in ['名詞', '形容詞', '動詞', '記号']:
            r.append(w)
    rl = (" ".join(r)).strip()
    results.append(rl)
    #print(rl) # --- 画面に分かち書きした行を表示

    # 書き込み先テキストを開く --- (※5)
wakati_file = '太宰治.wakati'
with open(wakati_file, 'w', encoding='utf-8') as fp:
    fp.write("\n".join(results))

# Word2Vecでモデルを作成 --- (※6)
data = word2vec.LineSentence(wakati_file)
model = word2vec.Word2Vec(data, 
    size=200, window=10, hs=1, min_count=2, sg=1)
model.save('太宰治.model')



In [141]:
#芥川龍之介
AR_URL = ['https://www.aozora.gr.jp/cards/000879/files/127_ruby_150.zip',
          'https://www.aozora.gr.jp/cards/000879/files/42377_ruby_34744.zip',
          'https://www.aozora.gr.jp/cards/000879/files/42_ruby_154.zip',
          'https://www.aozora.gr.jp/cards/000879/files/15_ruby_904.zip',
          'https://www.aozora.gr.jp/cards/000879/files/3804_ruby_27189.zip',
          'https://www.aozora.gr.jp/cards/000879/files/43017_ruby_17394.zip',
          'https://www.aozora.gr.jp/cards/000879/files/3803_ruby_27208.zip',
          'https://www.aozora.gr.jp/cards/000879/files/37_ruby_1265.zip',
          'https://www.aozora.gr.jp/cards/000879/files/3820_ruby_27216.zip',
          'https://www.aozora.gr.jp/cards/000879/files/60_ruby_821.zip',
          'https://www.aozora.gr.jp/cards/000879/files/43751_ruby_27860.zip',
          'https://www.aozora.gr.jp/cards/000879/files/155_ruby_1146.zip',
          'https://www.aozora.gr.jp/cards/000879/files/143_ruby_833.zip',
          'https://www.aozora.gr.jp/cards/000879/files/166_ruby_1459.zip',
          'https://www.aozora.gr.jp/cards/000879/files/170_ruby_348.zip',
          'https://www.aozora.gr.jp/cards/000879/files/1141_ruby_5097.zip'] 

for i in AR_URL:
    download_file = download(i)
    text = convert(download_file)
    AR_text += text  

In [113]:
t = Tokenizer()
results = []
# テキストを一行ずつ処理する
lines = AR_text.split("\r\n")
for line in lines:
    s = line
    s = s.replace('｜', '')
    s = re.sub(r'《.+?》', '', s) # ルビを削除
    s = re.sub(r'［＃.+?］', '', s) # 入力注を削除
    tokens = t.tokenize(s) # 形態素解析
    # 必要な語句だけを対象とする --- (※4)
    r = []
    for tok in tokens:
        if tok.base_form == "*": # 単語の基本系を採用
            w = tok.surface
        else:
            w = tok.base_form
        ps = tok.part_of_speech # 品詞情報
        hinsi = ps.split(',')[0]
        if hinsi in ['名詞', '形容詞', '動詞', '記号']:
            r.append(w)
    rl = (" ".join(r)).strip()
    results.append(rl)
    #print(rl) # --- 画面に分かち書きした行を表示

    # 書き込み先テキストを開く --- (※5)
wakati_file = '芥川龍之介.wakati'
with open(wakati_file, 'w', encoding='utf-8') as fp:
    fp.write("\n".join(results))

# Word2Vecでモデルを作成 --- (※6)
data = word2vec.LineSentence(wakati_file)
model = word2vec.Word2Vec(data, 
    size=200, window=10, hs=1, min_count=2, sg=1)
model.save('芥川龍之介.model')

model = word2vec.Word2Vec.load('芥川龍之介.model')



In [185]:
#森鷗外
MO_URL = ['https://www.aozora.gr.jp/cards/000129/files/58126_ruby_73643.zip',
          'https://www.aozora.gr.jp/cards/000129/files/45245_ruby_21882.zip',
          'https://www.aozora.gr.jp/cards/000129/files/695_ruby_22805.zip',
          'https://www.aozora.gr.jp/cards/000129/files/45224_ruby_19890.zip',
          'https://www.aozora.gr.jp/cards/000129/files/2522_ruby_5001.zip',
          'https://www.aozora.gr.jp/cards/000129/files/689_ruby_23256.zip',
          'https://www.aozora.gr.jp/cards/000129/files/673_ruby_23254.zip',
          'https://www.aozora.gr.jp/cards/000129/files/2058_ruby_19627.zip',
          'https://www.aozora.gr.jp/cards/000129/files/694_ruby_23249.zip',
          'https://www.aozora.gr.jp/cards/000129/files/43030_ruby_17344.zip',
          'https://www.aozora.gr.jp/cards/000356/files/2060_ruby_23189.zip',
          'https://www.aozora.gr.jp/cards/000129/files/1054_ruby_23199.zip',
          'https://www.aozora.gr.jp/cards/000129/files/2298_ruby_6609.zip',
          'https://www.aozora.gr.jp/cards/000129/files/2084_ruby_17360.zip',
          'https://www.aozora.gr.jp/cards/000129/files/3614_ruby_12061.zip',
          'https://www.aozora.gr.jp/cards/000129/files/2082_ruby_23122.zip'] 

for i in MO_URL:
    download_file = download(i)
    text = convert(download_file)
    MO_text += text

In [186]:
t = Tokenizer()
results = []
# テキストを一行ずつ処理する
lines = MO_text.split("\r\n")
for line in lines:
    s = line
    s = s.replace('｜', '')
    s = re.sub(r'《.+?》', '', s) # ルビを削除
    s = re.sub(r'［＃.+?］', '', s) # 入力注を削除
    tokens = t.tokenize(s) # 形態素解析
    # 必要な語句だけを対象とする --- (※4)
    r = []
    for tok in tokens:
        if tok.base_form == "*": # 単語の基本系を採用
            w = tok.surface
        else:
            w = tok.base_form
        ps = tok.part_of_speech # 品詞情報
        hinsi = ps.split(',')[0]
        if hinsi in ['名詞', '形容詞', '動詞', '記号']:
            r.append(w)
    rl = (" ".join(r)).strip()
    results.append(rl)
    #print(rl) # --- 画面に分かち書きした行を表示

    # 書き込み先テキストを開く --- (※5)
wakati_file = '森鷗外.wakati'
with open(wakati_file, 'w', encoding='utf-8') as fp:
    fp.write("\n".join(results))

# Word2Vecでモデルを作成 --- (※6)
data = word2vec.LineSentence(wakati_file)
model = word2vec.Word2Vec(data, 
    size=200, window=10, hs=1, min_count=2, sg=1)
model.save('森鷗外.model')

model = word2vec.Word2Vec.load('森鷗外.model')

**3.結果**

夏目漱石

In [194]:
model = word2vec.Word2Vec.load('夏目漱石.model')
for word in ['結婚', '人生', '友人', '愛', '夢', '生きる', '他人']:
    words = model.most_similar(positive=[word])
    n = [w[0] for w in words]
    print(word,"=", ",".join(n))

結婚 = 生まれつく,集散,承諾,諾否,破約,予審,祝,判事,打明ける,臆測
人生 = 進捗,争論,口舌,願望,観,嗜欲,有意義,閑生涯,原動力,暗示
友人 = 奉職,だする,バルザック,告別,横浜,一面識,吐月峰,出帆,語り物,用達
愛 = 芸妓,男性,都会人,カ条,権能,渝,方案,完,睦む,永遠
夢 = うなされる,くるしい,冷汗,うずく,空模様,ダイヤモンド,慧,絶間,五彩,空山
生きる = 曲淵,耶蘇孔,張付ける,理性,囁く,誨,ちょん髷,慈悲,いるか,かこつ
他人 = 傾城,りょう,推して知るべし,親身,朝晩,容喙,あか,妄り,慈愛,敵討


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [193]:
model = word2vec.Word2Vec.load('夏目漱石.model')

iwords = ['他人','社会','人生' ]
words = model.most_similar(positive=iwords)
n = [w[0] for w in words]
print(words,"=", ",".join(n))

[('切り捨てる', 0.7673225998878479), ('原動力', 0.7617892026901245), ('切り捨て', 0.7449092864990234), ('たかれる', 0.7423896193504333), ('有意義', 0.7422025799751282), ('天分', 0.7308719754219055), ('本末', 0.7267889976501465), ('口舌', 0.7266632914543152), ('今夕', 0.7227792739868164), ('暗示', 0.7209005355834961)] = 切り捨てる,原動力,切り捨て,たかれる,有意義,天分,本末,口舌,今夕,暗示


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  after removing the cwd from sys.path.


太宰治

In [192]:
model = word2vec.Word2Vec.load('太宰治.model')
for word in ['結婚', '人生', '友人', '愛', '夢', '生きる', '他人']:
    words = model.most_similar(positive=[word])
    n = [w[0] for w in words]
    print(word,"=", ",".join(n))

結婚 = 不承知,営む,内福,ときめかす,誤解,初婚,世継,生き抜く,推す,著述
人生 = 指針,既婚,切実,かがやかしい,受難,片すみ,二目,のる,懐疑,典雅
友人 = 八つ手,ピクニツク,でかける,承認,けち,仲直り,散髪,卑怯,構える,辛抱
愛 = 禽家,禽,誠,誤解,模倣,偉大,とぼしい,血液,サタン,独創
夢 = 五臓,離れ島,仮寝,エヘッ,きだ,空しい,醒める,つづ,慕,うなされる
生きる = 生きのびる,死ねる,成り行き,考え方,定法,栄冠,たえる,朗らか,事業,無くす
他人 = 真顔,思いやる,心持,難易,恋しい,監獄,ゆき,悩む,報恩,不可解


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [191]:
model = word2vec.Word2Vec.load('太宰治.model')

iwords = ['他人','社会','人生' ]
words = model.most_similar(positive=iwords)
n = [w[0] for w in words]
print(words,"=", ",".join(n))

[('真顔', 0.667510986328125), ('のる', 0.6629472374916077), ('転嫁', 0.6621818542480469), ('切実', 0.6575527191162109), ('既婚', 0.6326800584793091), ('二目', 0.6281033754348755), ('高慢ちき', 0.619822084903717), ('無頼', 0.6193315386772156), ('三目', 0.6191238164901733), ('知ったかぶり', 0.609961211681366)] = 真顔,のる,転嫁,切実,既婚,二目,高慢ちき,無頼,三目,知ったかぶり


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  after removing the cwd from sys.path.


芥川龍之介

In [190]:
model = word2vec.Word2Vec.load('芥川龍之介.model')
for word in ['結婚', '人生', '友人', '愛', '夢', '生きる', '他人']:
    words = model.most_similar(positive=[word])
    n = [w[0] for w in words]
    print(word,"=", ",".join(n))

結婚 = 調節,披露,有効,性慾,飛びこむ,つらなる,相談,式,庭前,古池
人生 = 踏み入る,泳げる,主催,大会,創痍,競技,オリムピツク,胎内,処する,踏み止まる
友人 = 金持ち,甲,児女,乙,親密,痛痒,未開,人種,打ち倒す,虚偽
愛 = 憐憫,適す,一半,公,高尚,悪党,必,神聖,養育,利己
夢 = 蓄,怒り,獅子王,革命,億,いく,薄れる,哩,風の音,仄めく
生きる = ゆく,枯らす,うせる,あたり前,未練,思える,釈,苦しい,現在,ほど
他人 = かつ,敏感,談,傍若無人,俗人,グルモン,一半,絶縁,名士,鹹水


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [189]:
model = word2vec.Word2Vec.load('芥川龍之介.model')

iwords = ['他人','社会','人生' ]
words = model.most_similar(positive=iwords)
n = [w[0] for w in words]
print(words,"=", ",".join(n))

[('石器', 0.8750275373458862), ('鋭敏', 0.8633654117584229), ('終止', 0.8608265519142151), ('極度', 0.8605843782424927), ('ベエメ', 0.8572107553482056), ('スウエデンボルグ', 0.8549501895904541), ('衰退', 0.8534131050109863), ('脳髄', 0.8521925806999207), ('亡国', 0.8503350615501404), ('長足', 0.8472084999084473)] = 石器,鋭敏,終止,極度,ベエメ,スウエデンボルグ,衰退,脳髄,亡国,長足


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  after removing the cwd from sys.path.


森鷗外

In [187]:
model = word2vec.Word2Vec.load('森鷗外.model')
for word in ['結婚', '人生', '友人', '愛', '夢', '生きる', '他人']:
    words = model.most_similar(positive=[word])
    n = [w[0] for w in words]
    print(word,"=", ",".join(n))

結婚 = dub,余計,考え直す,疎む,落第,枯らす,騎士,卑賤,択ぶ,多言
人生 = 日常,Apollon,利他,押し広める,色調,飛躍,忠義,外れる,利己,social
友人 = 其墓,在京,其婚姻,其年,竜華,二本松,興る,淇園,往訪,其伝記
愛 = 憂,没,是清,阿武,逸書,滝の川,満堂,堪る,吟行,西土
夢 = 浮ぶ,美しい,過ぎ去る,梅暦,見掛ける,人情本,奩体,縁遠い,蝦蟇,治郎
生きる = 機縁,復讐,無くなる,専売,アンドレア・バルヂピエロ,厄介,老衰,過度,迫害,すねる
他人 = 決断,制する,順境,感慨,かかわる,通達,身一つ,侮る,けんか,美貌


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [188]:
model = word2vec.Word2Vec.load('森鷗外.model')

iwords = ['他人','社会','人生' ]
words = model.most_similar(positive=iwords)
n = [w[0] for w in words]
print(words,"=", ",".join(n))

[('肯定', 0.8774096965789795), ('ルソオ', 0.871634304523468), ('social', 0.8628804087638855), ('政策', 0.8583863973617554), ('忠義', 0.857492983341217), ('万有', 0.853191614151001), ('人工', 0.8511061668395996), ('訐', 0.8488544225692749), ('浅薄', 0.8481053113937378), ('Atavisme', 0.8436183929443359)] = 肯定,ルソオ,social,政策,忠義,万有,人工,訐,浅薄,Atavisme


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  after removing the cwd from sys.path.


**4.考察**

四者の作風が垣間見えるようになり、その違いが浮かび上がるようになった。
他人に対する類似単語からその違いが最も分かる。

夏目漱石：傾城,りょう,推して知るべし,親身,朝晩,容喙,あか,妄り,慈愛,敵討

太宰治：真顔,思いやる,心持,難易,恋しい,監獄,ゆき,悩む,報恩,不可解

芥川龍之介：かつ,敏感,談,傍若無人,俗人,グルモン,一半,絶縁,名士,鹹水

森鷗外：決断,制する,順境,感慨,かかわる,通達,身一つ,侮る,けんか,美貌

夏目漱石は親身、慈愛など、有効な間柄であるとともに敵討という不穏なものも孕んでいる。

太宰治は不可解、難易、監獄、悩むと、他人を理解できない心情が表れている。

芥川龍之介は敏感、傍若無人、俗人、絶縁と、他人と相容れない人間であることが分かる。

森鷗外は他人を競争相手であったり、敵であるという考えがあるように思える。

しかし、どれも類似単語とは言えそうもない単語が半分程度ある。一作品の影響を、まだまだ強く受けているので、より作品数を増やさなければ、正確には捉えられない。


参考文献
https://qiita.com/y_itoh/items/1e91ec59dac403d8e8e6　：　Pythonによる自然言語処理
